In [1]:
import torch
device = torch.device("mps")

import evaluate
import sacrebleu
from tqdm import tqdm

from datasets import Dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments

/opt/miniconda3/envs/work/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/miniconda3/envs/work/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <843938F4-8FEE-3058-B0A3-50B73FAF02AB> /opt/miniconda3/envs/work/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/opt/miniconda3/envs/work/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/miniconda3/envs/work/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/miniconda3/envs/work/lib/python3.9/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/miniconda3/envs/work/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` i

### Part 1) Loading the finetuned model, test and benchmark datasets


In [40]:
# function to prepare the training and any other dataset
def prepare_data(eng_file_path, mai_file_path, data_type):
    # load the english and maithili texts files
    with open(eng_file_path, "r", encoding="utf-8") as en_file:
        eng_texts = en_file.readlines()

    with open(mai_file_path, "r", encoding="utf-8") as maithili_file:
        mai_texts = maithili_file.readlines()

    assert len(eng_texts) == len(mai_texts), "The number of sentences in both files must be the same."

    # clean the text files
    eng_texts_cleaned = [text.strip() for text in eng_texts]
    mai_texts_cleaned = [text.strip() for text in mai_texts]

    # create the dataset
    data = {
        "source_text": eng_texts_cleaned,
        "target_text": mai_texts_cleaned, 
    }
    dataset = Dataset.from_dict(data)

    # split the dataset into train, validation and test sets
    if data_type == "train":
        train_dataset, temp_dataset = dataset.train_test_split(test_size=0.1).values()
        val_dataset, test_dataset = temp_dataset.train_test_split(test_size=0.5).values()

        print(f"Training set size: {len(train_dataset)}")
        print(f"Validation set size: {len(val_dataset)}")
        print(f"Test set size: {len(test_dataset)}")

        return train_dataset, val_dataset, test_dataset
    elif data_type == "test":
        return dataset

# preprocessor function for tokenizer
def preprocess_function(examples, tokenizer):
    inputs = tokenizer(examples["source_text"], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer(examples["target_text"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

# function to tokenize the data
def tokenize_dataset(dataset, tokenizer):
    dataset_tokenized = dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
    return dataset_tokenized

# function to move the dataset to device
def move_to_device(batch):
    # move each tensor in the batch to the MPS device
    for key in batch:
        batch[key] = torch.tensor(batch[key]).to(device)
    return batch

# function to batch predict the model
def trainer_evaluate(model, tokenizer, test_dataset):

    eval_trainer = Seq2SeqTrainer(
        model=model,
        args = Seq2SeqTrainingArguments(
            output_dir="./results/test/",
            per_device_eval_batch_size=32,
            predict_with_generate=True,
            disable_tqdm=False,
        ), 
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
    )

    return eval_trainer

# function to evaluate the model
def compute_chrf(predictions, references):
    chrf = evaluate.load("chrf")
    chrf_score = chrf.compute(predictions=predictions, references=references, word_order=2)
    return chrf_score 

In [5]:
# load the model and tokenizer
model_path  = "./finetuned/epoch1"
finetuned_tokenizer = AutoTokenizer.from_pretrained(model_path) 
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
print("Parameter size:", finetuned_model.num_parameters())

# read test and benchmark data
test_dataset = prepare_data("./dataset/training_split/bpcc/test.eng_Latn", "./dataset/training_split/bpcc/test.mai_Deva", "test")

# choose random 1200 examples from test dataset for faster inference and evaluation
test_dataset = test_dataset.shuffle(seed=42).select(range(1200))
test_dataset_tokenized = tokenize_dataset(test_dataset, finetuned_tokenizer)

in22_mai_test = prepare_data("./dataset/test/IN22_test/gen/test.eng_Latn", "./dataset//test/IN22_test/gen/test.mai_Deva", "test")
in22_mai_test_tokenized = tokenize_dataset(in22_mai_test, finetuned_tokenizer)

print("Test dataset size:", len(test_dataset))
print("IN22 benchmark dataset size:", len(in22_mai_test))

Parameter size: 76676096


Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

Test dataset size: 1200
IN22 benchmark dataset size: 1024


### Part 2) Testing on BPCC Eng-Mai test split


In [6]:
# move the model and in22 testdata to the MPS device
finetuned_model = finetuned_model.to(device)
test_dataset_tokenized = test_dataset_tokenized.with_transform(move_to_device)

# generate predictions for english to hindi
eval_trainer = trainer_evaluate(finetuned_model, finetuned_tokenizer, test_dataset_tokenized)
test_dataset_mai_pred, test_dataset_mai_lab, _ = eval_trainer.predict(test_dataset_tokenized)

# decode the predictions and references
test_dataset_mai_pred = finetuned_tokenizer.batch_decode(test_dataset_mai_pred, skip_special_tokens=True)
test_dataset_mai_ref = finetuned_tokenizer.batch_decode(test_dataset_mai_lab, skip_special_tokens=True)

  0%|          | 0/38 [00:00<?, ?it/s]

In [42]:
# print the predictions and references for comparison
print("Testing results for test dataset\n")
for i in range(2):
    print("English Text : ", test_dataset[i]['source_text'])
    print("Maithili Reference: ", test_dataset_mai_ref[i])
    print("Maithili Prediction: ",test_dataset_mai_pred[i])
    print("\n")

# calculate chrF++ score for hindi to maithili overlap
chrf_score_mai = compute_chrf(test_dataset_mai_pred, test_dataset_mai_ref)
print(f"chrF++ score for English-Maithili test data split: {chrf_score_mai['score']}")

Testing results for test dataset

English Text :  However, the Chola dynasty seemed to have survived elsewhere outside of India.
Maithili Reference:  ओना भार बाहर आन ठाम चोल वंश बचि गेल बाइत छल।
Maithili Prediction:  एकार प्र्रार्रारा प्र्रारार स्रार्रारा स्रारारा स्रेलेलेल स्र्रेल लेलेलेलेल लेलेलेलेल प्रेलेलेलेलेलेलेलेल प्रेलेलेल प्रेलेलेल क लेलेलेलेलेलेलेलेलेलेलेल लेलेलेलेल।


English Text :  Rockport is a city in the U.S. state of Texas.
Maithili Reference:  रॉकपोर्ट सन््त राज्य अमेरिका क टेक्सास राज्य क एकटा र थी
Maithili Prediction:  एकार स्र्र्र्र्र्रा स्र्र्र्रारा स्र्रार्रा स्र्र्राल स्रेल स्रेल स्रेल स्रेल स्रेलेल स्रेलेलेलेल स्रेल स्रेलेल।


chrF++ score for English-Maithili test data split: 5.077240987101195


### Part 3) Testing on IN22 Eng-Mai benchmark dataset


In [43]:

# move the model and in22 testdata to the MPS device
finetuned_model = finetuned_model.to(device)
in22_mai_test_tokenized = in22_mai_test_tokenized.with_transform(move_to_device)

# generate predictions for english to hindi
eval_trainer = trainer_evaluate(finetuned_model, finetuned_tokenizer, in22_mai_test_tokenized)
in22_mai_test_pred, in22_mai_test_lab, _ = eval_trainer.predict(in22_mai_test_tokenized)

# decode the predictions and references
in22_mai_test_pred = finetuned_tokenizer.batch_decode(in22_mai_test_pred, skip_special_tokens=True)
in22_mai_test_ref = finetuned_tokenizer.batch_decode(in22_mai_test_lab, skip_special_tokens=True)

  0%|          | 0/32 [00:00<?, ?it/s]

In [45]:
# print the predictions and references for comparison
for i in range(2):
    print("English Text: ", in22_mai_test[i]["source_text"])
    print("Maithili Reference: ", in22_mai_test_ref[i])
    print("Maithili Prediction: ", in22_mai_test_pred[i])
    print("\n")

# calculate chrF++ score for hindi to maithili overlap
chrf_score_mai = compute_chrf(in22_mai_test_pred, in22_mai_test_ref)
print(f"chrF++ score for English-Maithili IN22 benchmark dataset: {chrf_score_mai['score']}")

English Text:  An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.
Maithili Reference:  रूप सर्ला व्यक् सम्बन् ब रास लक् सम होयत  जेना हुनक ा, कपड़ा, टाई, गहना, , श्रृंगार, ़ी, प्रसाधन सामग्री, सेंट इत्या
Maithili Prediction:  एकार प्र्र्रार्र्र्रा प्र्रार्र्रा प्र्र्र्र्र्र्र्र्र्र्रा पर्र्र्र्र्र्र्रा प्र्र्र्र्रार्र्र्रान स्र्र्र्र्र्रान पार्र्रान प्र्रेल, स्र्र्र्रेलेल प्रेलेलेलेलेल स्र्र्र्र्र्रेलेलेल, स्रेलेलेलेलेलेलेलेलेलेलेलेलेलेल, स्त्त्रेल, स्त्त्रेलेलेलेल, स्त्त्र्र्र्र्थ्र्रेल, स्रेल, लेलेल, लेल, लेलेलेलेलेलेल, पे पेलेलेलेलेलेलेल, लेल, लेल, लेलेलेलेलेलेल, लेलेलेलेलेलेलेल, लेल, लेलेलेलेल, लेलेल लेल, ल ल लेलेलेलेल ल आ लेलेलेलेल।


English Text:  Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.
Maithili Reference:  महाराष्ट